## Setup Azure ML

In [2]:
from azureml.core import Workspace

subscription_id = '<subscription-id>'
resource_group = '<resource-group>'
workspace_name = '<ml-workspace-name>'

try:
    ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
    ws.write_config()
    print("Workspace configuration succeeded")
except:
    print("Workspace not accessible")

Workspace configuration succeeded


## Fetch dataset

In [4]:
from azureml.core import Dataset

dataset = Dataset.get_by_name(ws, 'predictive_maintenance')

## Train model

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cpu_cluster_name = 'cpu-cluster'

try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS2_V2',
                                                           min_nodes=0,
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

In [ ]:
from azureml.train.automl import AutoMLConfig
import logging

automl_settings = {
    "n_cross_validations": 3,
    "primary_metric": 'average_precision_score_weighted',
    "enable_early_stopping": True,
    "max_concurrent_iterations": 1,
    "experiment_timeout_hours": 0.25,
    "verbosity": logging.INFO,
    "iterations": 10,
    "enable_onnx_compatible_models": True
}

automl_config = AutoMLConfig(task = 'classification',
                             compute_target = compute_target,
                             debug_log = 'automl_errors.log',
                             training_data = dataset,
                             label_column_name = 'Status',
                             **automl_settings)

In [ ]:
from azureml.core.experiment import Experiment

experiment = Experiment(ws, 'predictive-maintenance')
run = experiment.submit(automl_config, show_output = False)

In [ ]:
from azureml.widgets import RunDetails
RunDetails(run).show()

In [ ]:
run.wait_for_completion()

## Prepare model

In [ ]:
run.register_model(model_name = 'predictive_maintenance')

In [ ]:
best_run, fitted_model = run.get_output(return_onnx_model=True)
print(best_run)
print(fitted_model)

In [ ]:
from azureml.automl.runtime.onnx_convert import OnnxConverter

onnx_fl_path = "./model.onnx"
OnnxConverter.save_onnx_model(fitted_model, onnx_fl_path)